In [18]:
# Cài đặt spaCy
!pip install -U spacy

# Tải về mô hình tiếng Anh
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 13.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [19]:
import spacy
from spacy import displacy

## Phần 2: Phân tích câu và Trực quan hóa

Trực quan hóa là cách tốt nhất để bắt đầu hiểu về cây phụ thuộc. spaCy cung cấp một công cụ tuyệt vời tên là displaCy.

### 2.1. Tải mô hình và phân tích câu

In [20]:
nlp = spacy.load("en_core_web_md")

# Câu ví dụ
text = "The quick brown fox jumps over the lazy dog."

# Phân tích câu với pipeline của spaCy
doc = nlp(text)

In [21]:
print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 70)

for token in doc:
    children = [child.text for child in token.children]
    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")

TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
The          | det        | fox          | NOUN     | []
quick        | amod       | fox          | NOUN     | []
brown        | amod       | fox          | NOUN     | []
fox          | nsubj      | jumps        | VERB     | ['The', 'quick', 'brown']
jumps        | ROOT       | jumps        | VERB     | ['fox', 'over', '.']
over         | prep       | jumps        | VERB     | ['dog']
the          | det        | dog          | NOUN     | []
lazy         | amod       | dog          | NOUN     | []
dog          | pobj       | over         | ADP      | ['the', 'lazy']
.            | punct      | jumps        | VERB     | []


In [22]:
# Tùy chọn để hiển thị trong trình duyệt
# options = {"compact": True, "color": "blue", "font": "Source Sans Pro"}
# Khởi chạy server tại http://127.0.0.1:5000
# Bạn có thể truy cập địa chỉ này trên trình duyệt để xem cây phụ thuộc
# Nhấn Ctrl+C trong terminal để dừng server
# displacy.serve(doc, style="dep")

# Hoặc hiển thị trong notebook
displacy.render(doc, style="dep", jupyter=True, options={"distance": 100})

**Câu hỏi:**
- Từ nào là gốc (ROOT) của câu?
- jumps có những từ phụ thuộc (dependent) nào? Các quan hệ đó là gì?
- fox là head của những từ nào?


## Phần 3: Truy cập các thành phần trong cây phụ thuộc

Trực quan hóa rất hữu ích, nhưng sức mạnh thực sự đến từ việc truy cập cây phụ thuộc theo chương trình. Mỗi Token trong đối tượng Doc của spaCy chứa đầy đủ thông tin về vị trí của nó trong cây.

In [23]:
text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text)

print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 70)

for token in doc:
    children = [child.text for child in token.children]
    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")

TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | ['Apple', 'is', 'at']
at           | prep       | looking      | VERB     | ['buying']
buying       | pcomp      | at           | ADP      | ['startup']
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | ['U.K.', 'for']
for          | prep       | startup      | NOUN     | ['billion']
$            | quantmod   | billion      | NUM      | []
1            | compound   | billion      | NUM      | []
billion      | pobj       | for          | ADP      | ['$', '1']


**Giải thích các thuộc tính:**
- `token.text`: Văn bản của token.
- `token.dep_`: Nhãn quan hệ phụ thuộc của token này với head của nó.
- `token.head.text`: Văn bản của token head.
- `token.head.pos_`: Part-of-Speech tag của token head.
- `token.children`: Một iterator chứa các token con (dependent) của token hiện tại.

## Phần 4: Duyệt cây phụ thuộc để trích xuất thông tin

### 4.1. Bài toán: Tìm chủ ngữ và tân ngữ của một động từ

In [24]:
text = "The cat chased the mouse and the dog watched them."
doc = nlp(text)

for token in doc:
    # Chỉ tìm các động từ
    if token.pos_ == "VERB":
        verb = token.text
        subject = ""
        obj = ""
        
        # Tìm chủ ngữ (nsubj) và tân ngữ (dobj) trong các con của động từ
        for child in token.children:
            if child.dep_ == "nsubj":
                subject = child.text
            if child.dep_ == "dobj":
                obj = child.text
        
        if subject and obj:
            print(f"Found Triplet: ({subject}, {verb}, {obj})")

Found Triplet: (cat, chased, mouse)
Found Triplet: (dog, watched, them)


### 4.2. Bài toán: Tìm các tính từ bổ nghĩa cho một danh từ

In [ ]:
text = "The big, fluffy white cat is sleeping on the warm mat."
doc = nlp(text)

for token in doc:
    # Chỉ tìm các danh từ
    if token.pos_ == "NOUN":
        adjectives = []
        
        # Tìm các tính từ bổ nghĩa (amod) trong các con của danh từ
        for child in token.children:
            if child.dep_ == "amod":
                adjectives.append(child.text)
        
        if adjectives:
            print(f"Danh từ '{token.text}' được bổ nghĩa bởi các tính từ: {adjectives}")

---

## Phần 5: Bài tập tự luyện

### Bài 1: Tìm động từ chính của câu

Động từ chính của câu thường có quan hệ ROOT. Viết một hàm `find_main_verb(doc)` nhận vào một đối tượng Doc của spaCy và trả về Token là động từ chính.

In [ ]:
def find_main_verb(doc):
    """
    Tìm động từ chính của câu (có quan hệ ROOT)
    """
    for token in doc:
        if token.dep_ == "ROOT" and token.pos_ == "VERB":
            return token
    return None

# Test hàm
test_sentence = "The student studied hard and passed the exam."
doc_test = nlp(test_sentence)
main_verb = find_main_verb(doc_test)
if main_verb:
    print(f"Động từ chính: {main_verb.text}")
else:
    print("Không tìm thấy động từ chính")

### Bài 2: Trích xuất các cụm danh từ (Noun Chunks)

spaCy đã có sẵn thuộc tính `.noun_chunks` để trích xuất các cụm danh từ. Tuy nhiên, hãy thử tự viết một hàm để làm điều tương tự.

**Gợi ý:** Một cụm danh từ đơn giản là một danh từ và tất cả các từ bổ nghĩa cho nó (như det, amod, compound). Bạn có thể bắt đầu từ một danh từ và duyệt ngược lên head hoặc duyệt xuống các children của nó.

In [ ]:
def extract_noun_chunks(doc):
    """
    Trích xuất các cụm danh từ từ doc
    """
    noun_chunks = []
    
    for token in doc:
        # Tìm các danh từ là head của cụm danh từ
        if token.pos_ == "NOUN":
            # Thu thập các từ bổ nghĩa (det, amod, compound)
            chunk_tokens = []
            
            # Thêm các từ bổ nghĩa (children với dep là det, amod, compound)
            for child in token.children:
                if child.dep_ in ["det", "amod", "compound"]:
                    chunk_tokens.append(child)
            
            # Sắp xếp theo thứ tự trong câu
            chunk_tokens.append(token)
            chunk_tokens.sort(key=lambda t: t.i)
            
            # Tạo chuỗi cụm danh từ
            chunk_text = " ".join([t.text for t in chunk_tokens])
            noun_chunks.append(chunk_text)
    
    return noun_chunks

# Test hàm
test_sentence = "The big, fluffy white cat is sleeping on the warm mat."
doc_test = nlp(test_sentence)
chunks = extract_noun_chunks(doc_test)
print("Các cụm danh từ tìm được:")
for chunk in chunks:
    print(f"  - {chunk}")

# So sánh với kết quả của spaCy
print("\nKết quả từ spaCy noun_chunks:")
for chunk in doc_test.noun_chunks:
    print(f"  - {chunk.text}")

### Bài 3: Tìm đường đi ngắn nhất trong cây

Viết một hàm `get_path_to_root(token)` để tìm đường đi từ một token bất kỳ lên đến gốc (ROOT) của cây. Hàm nên trả về một danh sách các token trên đường đi.

In [ ]:
def get_path_to_root(token):
    """
    Tìm đường đi từ một token lên đến ROOT
    """
    path = [token]
    current = token
    
    # Đi ngược lên head cho đến khi gặp ROOT
    while current.head != current:
        current = current.head
        path.append(current)
        # Dừng nếu đã đến ROOT
        if current.dep_ == "ROOT":
            break
    
    return path

# Test hàm
test_sentence = "The quick brown fox jumps over the lazy dog."
doc_test = nlp(test_sentence)

# Tìm đường đi từ "dog" lên ROOT
dog_token = None
for token in doc_test:
    if token.text == "dog":
        dog_token = token
        break

if dog_token:
    path = get_path_to_root(dog_token)
    print(f"Đường đi từ '{dog_token.text}' lên ROOT:")
    for i, token in enumerate(path):
        print(f"  {i+1}. {token.text} ({token.dep_})")

---

## Tổng kết

Trong bài thực hành này, chúng ta đã đi từ những khái niệm cơ bản đến các ứng dụng thực tế của phân tích cú pháp phụ thuộc. Bạn đã học cách sử dụng spaCy để phân tích câu, trực quan hóa kết quả, và quan trọng nhất là duyệt cây phụ thuộc để trích xuất các thông tin ngữ nghĩa quan trọng. Nắm vững kỹ thuật này sẽ mở ra rất nhiều khả năng cho các ứng dụng NLP phức tạp hơn.